## Imports

In [212]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import talib
from collections import Counter
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler
from ta.volume import ForceIndexIndicator

## Functions

In [213]:
def load_data(data_path):
    df = pd.read_csv(data_path)
    return df


def preprocess_data(df):
    # Create a 'next_color' column
    df["next_color"] = df["color"].shift(-1)

    # Create a 'color_change' column
    df["color_change"] = np.where(df["color"] != df["next_color"], 1, 0)

    # Drop the last row
    df = df[:-1]

    # Drop 'color' and 'next_color' columns
    df = df.drop(["color", "next_color"], axis=1)

    df = dynamic_line_break(df)

    return df


def check_data_balance(y):
    counter = Counter(y)
    for label, count in counter.items():
        percentage = (count / len(y)) * 100
        print(f"Class {label}: {count} samples ({percentage:.2f}%)")


def add_heiken_ashi_features(df):
    # Create Heiken Ashi DataFrame
    ha_df = df.ta.ha()

    # Rename the HA columns
    ha_df.columns = [f"HA_{col}" for col in ha_df.columns]

    # Join the HA columns to the original dataframe
    df = df.join(ha_df)

    # Heiken Ashi Close to Open
    df["HA_close_open"] = df["HA_close"] - df["HA_open"]

    # Heiken Ashi High Low Range
    df["HA_high_low"] = df["HA_high"] - df["HA_low"]

    # Heiken Ashi Body Range
    df["HA_body"] = abs(df["HA_close"] - df["HA_open"])

    # Heiken Ashi Price Direction
    df["HA_direction"] = (df["HA_close"] > df["HA_open"]).astype(int)

    # Heiken Ashi Volume-weighted Price
    df["HA_vwap"] = (df["HA_close"] * df["volume"]).cumsum() / df["volume"].cumsum()

    # Lag 1 feature
    df["HA_close_lag1"] = df["HA_close"].shift(1)

    # Close Change
    df["HA_close_change"] = df["HA_close"].diff()

    # Close % Change
    df["HA_close_pct_change"] = df["HA_close"].pct_change()

    # 5-period Simple Moving Average
    df["HA_sma5"] = df["HA_close"].rolling(5).mean()

    # 5-period Exponential Moving Average
    df["HA_ema5"] = df["HA_close"].ewm(span=5).mean()

    # Additional features
    df["HA_ema10"] = df["HA_close"].ewm(span=10).mean()
    df["HA_ema15"] = df["HA_close"].ewm(span=15).mean()
    df["HA_pct_diff_ema5_15"] = (
        (df["HA_ema5"] - df["HA_ema15"]) / df["HA_ema15"]
    ) * 100
    df["HA_rsi"] = ta.rsi(df["HA_close"])
    # Calculate Short Term Exponential Moving Average
    df["short_ema"] = df["HA_close"].ewm(span=12, adjust=False).mean()

    # Calculate Long Term Exponential Moving Average
    df["long_ema"] = df["HA_close"].ewm(span=26, adjust=False).mean()

    # Calculate Moving Average Convergence Divergence (MACD)
    df["HA_macd"] = df["short_ema"] - df["long_ema"]

    # Calculate Signal Line
    df["HA_macds"] = df["HA_macd"].ewm(span=9, adjust=False).mean()

    # Calculate MACD Histogram
    df["HA_macdh"] = df["HA_macd"] - df["HA_macds"]

    # Drop temporary short_ema and long_ema columns
    df = df.drop(["short_ema", "long_ema"], axis=1)

    df["HA_cci"] = ta.cci(df["HA_high"], df["HA_low"], df["HA_close"])
    df["HA_atr"] = ta.atr(df["HA_high"], df["HA_low"], df["HA_close"])
    df["HA_ha_close_bbp50_std"] = (
        ta.stdev(df["HA_close"], 50) / df["HA_close"]
    )  # Bollinger Bands normalized width
    df["HA_mfi"] = ta.mfi(df["HA_high"], df["HA_low"], df["HA_close"], df["volume"])

    return df


def compute_renko(df, timeperiod=14, multiplier=0.25):
    # Calculate ATR
    atr_values = talib.ATR(df["high"], df["low"], df["close"], timeperiod)

    # Drop rows with NaN ATR values
    df = df[atr_values.notna()]
    atr_values = atr_values.dropna()

    # Compute average ATR
    average_atr = atr_values.mean()

    # Set brick size
    brick_size = average_atr * multiplier

    renko_df = pd.DataFrame(
        index=df.index, columns=["open", "high", "low", "close", "time_unix"]
    )

    current_price = df["close"][0]
    last_reset_price = current_price

    for idx in df.index[1:]:
        current_price = df.loc[idx, "close"]
        renko_df.loc[idx, "time_unix"] = df.loc[
            idx, "time_unix"
        ]  # Copy 'time_unix' from df

        while last_reset_price + brick_size <= current_price:
            renko_df.loc[idx] = [
                last_reset_price,
                last_reset_price + brick_size,
                last_reset_price,
                last_reset_price + brick_size,
                df.loc[idx, "time_unix"],
            ]
            last_reset_price += brick_size

        while last_reset_price - brick_size >= current_price:
            renko_df.loc[idx] = [
                last_reset_price,
                last_reset_price - brick_size,
                last_reset_price - brick_size,
                last_reset_price,
                df.loc[idx, "time_unix"],
            ]
            last_reset_price -= brick_size

    return renko_df.dropna()


def add_renko_features(df, renko_df):
    # Append "_renko" to the column names of renko_df
    renko_df.columns = [str(col) + "_renko" for col in renko_df.columns]

    # Add EMA, SMA, WMA, RSI, and Bollinger Bands to renko_df
    renko_df["sma_renko"] = talib.SMA(renko_df["close_renko"], timeperiod=5)
    renko_df["ema_renko"] = talib.EMA(renko_df["close_renko"], timeperiod=5)
    renko_df["wma_renko"] = talib.WMA(renko_df["close_renko"], timeperiod=5)
    renko_df["rsi_renko"] = talib.RSI(renko_df["close_renko"], timeperiod=5)
    (
        renko_df["upper_band_renko"],
        renko_df["middle_band_renko"],
        renko_df["lower_band_renko"],
    ) = talib.BBANDS(renko_df["close_renko"], timeperiod=5)
    # Add ROC and Momentum to renko_df
    renko_df["roc_renko"] = talib.ROC(renko_df["close_renko"], timeperiod=5)
    renko_df["momentum_renko"] = renko_df["close_renko"].diff()

    # Add Stochastic Oscillator to renko_df
    renko_df["k_renko"], renko_df["d_renko"] = talib.STOCH(
        renko_df["high_renko"],
        renko_df["low_renko"],
        renko_df["close_renko"],
        fastk_period=5,
        slowk_period=3,
        slowd_period=3,
    )
    df = df.join(
        renko_df.set_index("time_unix_renko"), on="time_unix"
    )  # Join Renko data with the original DataFrame

    # Derived features
    df["close_open_renko"] = df["close_renko"] - df["open_renko"]
    df["high_low_renko"] = df["high_renko"] - df["low_renko"]
    df["close_change_renko"] = df["close_renko"].diff()
    df["direction_renko"] = df["close_change_renko"].apply(lambda x: int(x > 0))
    df["direction_change"] = df["direction_renko"].diff().abs()

    return df


def kagi(df):
    kagi_data = []
    cur_col = {
        "kagi_dir": "up",
        "kagi_start": df.loc[df.index[0], "close"],
        "time_unix": df.loc[df.index[0], "time_unix"],
        "volume": df.loc[df.index[0], "volume"],
    }
    prev_kagi_start = df.loc[df.index[0], "close"]
    for index, row in df.iterrows():
        cp = row["close"]
        diff = cp - cur_col["kagi_start"]
        if cur_col["kagi_dir"] == "up":
            if diff < 0:
                cur_col["kagi_high"] = cur_col["kagi_start"]
                cur_col["kagi_low"] = prev_kagi_start
                kagi_data.append(cur_col.copy())
                cur_col["kagi_dir"] = "down"
        else:
            if diff > 0:
                cur_col["kagi_low"] = cur_col["kagi_start"]
                cur_col["kagi_high"] = prev_kagi_start
                kagi_data.append(cur_col.copy())
                cur_col["kagi_dir"] = "up"
        prev_kagi_start = cur_col["kagi_start"]
        cur_col["kagi_start"] = cp
        cur_col["time_unix"] = row["time_unix"]
        cur_col["volume"] = row["volume"]
    cur_col["kagi_high"] = (
        cur_col["kagi_start"] if cur_col["kagi_dir"] == "up" else prev_kagi_start
    )
    cur_col["kagi_low"] = (
        cur_col["kagi_start"] if cur_col["kagi_dir"] == "down" else prev_kagi_start
    )
    kagi_data.append(cur_col)
    kagi_df = pd.DataFrame(kagi_data)

    # Convert 'kagi_dir' to numerical
    kagi_df["kagi_dir"] = kagi_df["kagi_dir"].map({"up": 1, "down": 0})

    return kagi_df


def add_kagi_features(df, kagi_df):
    # Calculate simple moving averages
    kagi_df["kagi_sma5"] = talib.SMA(kagi_df["kagi_start"], timeperiod=5)
    kagi_df["kagi_sma10"] = talib.SMA(kagi_df["kagi_start"], timeperiod=10)
    kagi_df["kagi_sma20"] = talib.SMA(kagi_df["kagi_start"], timeperiod=20)

    # Calculate Bollinger Bands
    kagi_df["kagi_upper"], kagi_df["kagi_middle"], kagi_df["kagi_lower"] = talib.BBANDS(
        kagi_df["kagi_start"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0
    )

    # Calculate MACD
    (
        kagi_df["kagi_macd"],
        kagi_df["kagi_macdsignal"],
        kagi_df["kagi_macdhist"],
    ) = talib.MACD(kagi_df["kagi_start"], fastperiod=12, slowperiod=26, signalperiod=9)

    # Calculate RSI
    kagi_df["kagi_rsi"] = talib.RSI(kagi_df["kagi_start"], timeperiod=14)

    # Calculate Stochastic Oscillator
    kagi_df["kagi_slowk"], kagi_df["kagi_slowd"] = talib.STOCH(
        kagi_df["kagi_high"],
        kagi_df["kagi_low"],
        kagi_df["kagi_start"],
        5,
        3,
        0,
        3,
        0,
    )

    kagi_df["kagi_atr"] = talib.ATR(
        kagi_df["kagi_high"],
        kagi_df["kagi_low"],
        kagi_df["kagi_start"],
        timeperiod=14,
    )

    kagi_df["kagi_adx"] = talib.ADX(
        kagi_df["kagi_high"],
        kagi_df["kagi_low"],
        kagi_df["kagi_start"],
        timeperiod=14,
    )

    # Convert df.index to Unix timestamp
    df.index = df.index.astype(np.int64) // 10**9

    # Align the volume series with kagi_df
    aligned_volume = df["volume"].reindex(kagi_df["time_unix"], method="pad")

    # Calculate Accumulation / Distribution Line
    kagi_df["kagi_ad"] = talib.AD(
        kagi_df["kagi_high"],
        kagi_df["kagi_low"],
        kagi_df["kagi_start"],
        aligned_volume,
    )

    # Rename volume to avoid overlapping when joining
    kagi_df.rename(columns={"volume": "kagi_volume"}, inplace=True)

    return kagi_df


def three_line_break(df):
    df_tlb = pd.DataFrame()
    df_tlb["open"] = df["open"].reset_index(drop=True)
    df_tlb["close"] = df["close"].reset_index(drop=True)
    df_tlb["high"] = df["high"].reset_index(drop=True)
    df_tlb["low"] = df["low"].reset_index(drop=True)
    df_tlb["volume"] = df["volume"].reset_index(drop=True)
    df_tlb["tlb_direction"] = None
    df_tlb["time_unix"] = df[
        "time_unix"
    ].values  # Copy 'time_unix' from original dataframe

    if df_tlb["close"][1] > df_tlb["close"][0]:
        df_tlb.loc[1, "tlb_direction"] = "up"
    else:
        df_tlb.loc[1, "tlb_direction"] = "down"

    for i in range(2, len(df_tlb)):
        up_condition = (df_tlb["close"][i] > df_tlb["close"][i - 3 : i].max()) and (
            df_tlb["tlb_direction"][i - 1] == "down"
        )
        down_condition = (df_tlb["close"][i] < df_tlb["close"][i - 3 : i].min()) and (
            df_tlb["tlb_direction"][i - 1] == "up"
        )

        if up_condition:
            df_tlb.loc[i, "tlb_direction"] = "up"
        elif down_condition:
            df_tlb.loc[i, "tlb_direction"] = "down"
        else:
            df_tlb.loc[i, "tlb_direction"] = df_tlb.loc[i - 1, "tlb_direction"]

    df_tlb["tlb_direction"] = df_tlb["tlb_direction"].map({"up": 1, "down": 0})

    return df_tlb


def add_tlb_features(df):
    # Add Relative Strength Index (RSI) of the close price
    df["tlb_RSI"] = talib.RSI(df["close"], timeperiod=14)

    # Add Moving Average Convergence Divergence (MACD) of the close price
    df["tlb_MACD"], df["tlb_MACD_signal"], df["tlb_MACD_hist"] = talib.MACD(
        df["close"], fastperiod=12, slowperiod=26, signalperiod=9
    )

    # Add Bollinger Bands of the close price
    df["tlb_upper_band"], df["tlb_middle_band"], df["tlb_lower_band"] = talib.BBANDS(
        df["close"], timeperiod=20
    )

    # Add Simple Moving Average (SMA) of the close price
    df["tlb_SMA"] = talib.SMA(df["close"], timeperiod=14)

    # Add Exponential Moving Average (EMA) of the close price
    df["tlb_EMA"] = talib.EMA(df["close"], timeperiod=14)

    # Add Rate of Change (ROC) of the close price
    df["tlb_ROC"] = talib.ROC(df["close"], timeperiod=10)

    # Add Average True Range (ATR) of the close price
    df["tlb_ATR"] = talib.ATR(df["high"], df["low"], df["close"], timeperiod=14)

    # Add Momentum of the close price
    df["tlb_momentum"] = talib.MOM(df["close"], timeperiod=10)

    # Add Stochastic Oscillator %K and %D of the close price
    df["tlb_slowk"], df["tlb_slowd"] = talib.STOCH(
        df["high"],
        df["low"],
        df["close"],
        fastk_period=5,
        slowk_period=3,
        slowk_matype=0,
        slowd_period=3,
        slowd_matype=0,
    )

    # Add Commodity Channel Index (CCI) of the close price
    df["tlb_CCI"] = talib.CCI(df["high"], df["low"], df["close"], timeperiod=14)

    # Add On Balance Volume (OBV)
    df["tlb_OBV"] = talib.OBV(df["close"], df["volume"])

    # Add Moving Average of High, Low, Open prices
    df["tlb_MA_high"] = talib.MA(df["high"], timeperiod=14)
    df["tlb_MA_low"] = talib.MA(df["low"], timeperiod=14)
    df["tlb_MA_open"] = talib.MA(df["open"], timeperiod=14)

    # Add Historical volatility
    df["tlb_volatility"] = (
        talib.ATR(df["high"], df["low"], df["close"], timeperiod=14) / df["tlb_SMA"]
    )

    # Add Money Flow Index (MFI)
    df["tlb_MFI"] = talib.MFI(
        df["high"], df["low"], df["close"], df["volume"], timeperiod=14
    )

    # Add Chaikin Money Flow (CMF)
    df["tlb_CMF"] = (
        df["close"] - df["low"] - (df["high"] - df["close"]) / (df["high"] - df["low"])
    ) * df["volume"]

    # Add William’s %R
    df["tlb_WilliamsR"] = talib.WILLR(df["high"], df["low"], df["close"], timeperiod=14)

    # Add Ultimate Oscillator
    df["tlb_UO"] = talib.ULTOSC(
        df["high"],
        df["low"],
        df["close"],
        timeperiod1=7,
        timeperiod2=14,
        timeperiod3=28,
    )

    # Add Accumulation/Distribution Line (ADL)
    df["tlb_ADL"] = talib.AD(df["high"], df["low"], df["close"], df["volume"])

    # Add Average Directional Index (ADX)
    df["tlb_ADX"] = talib.ADX(df["high"], df["low"], df["close"], timeperiod=14)

    return df


def custom_atr(df, period):
    high_low = df["high"] - df["low"]
    high_close = (df["high"] - df["close"].shift()).abs()
    low_close = (df["low"] - df["close"].shift()).abs()
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr


def atr(df, period):
    atr = ta.volatility.AverageTrueRange(
        df["high"], df["low"], df["close"], window=period
    )
    return atr.average_true_range()


def pnf(df, box_size, reversal):
    df["pnf_high"] = np.nan
    df["pnf_low"] = np.nan
    df["pnf_direction"] = 0
    df["pnf_dtb"] = 0
    df["pnf_dbb"] = 0
    df["pnf_num_reversals"] = 0
    df["pnf_avg_column_height"] = np.nan
    df["pnf_max_column_height"] = np.nan
    df["pnf_min_column_height"] = np.nan
    df["pnf_std_column_height"] = np.nan
    df["pnf_bullish_percentage"] = np.nan
    df["pnf_bearish_percentage"] = np.nan
    df["pnf_num_bullish_to_bearish_reversals"] = 0
    df["pnf_num_bearish_to_bullish_reversals"] = 0
    df["pnf_avg_bullish_run_length"] = np.nan

    column_heights = []
    bullish_run_lengths = []
    num_bullish_periods = 0
    num_bearish_periods = 0
    current_bullish_run_length = 0

    current_direction = 0
    current_high = df["high"].iloc[0]
    current_low = df["low"].iloc[0]
    last_high = current_high
    last_low = current_low

    for i in range(1, len(df)):
        if df["high"].iloc[i] >= current_high + box_size:
            current_direction = 1
            last_high = current_high
            current_high = df["high"].iloc[i]
            df.loc[df.index[i], "pnf_high"] = current_high
            df.loc[df.index[i], "pnf_direction"] = current_direction
            if last_high is not None and current_high > last_high:
                df.loc[df.index[i], "pnf_dtb"] = 1  # double top breakout
            column_heights.append(current_high - last_high)
            current_bullish_run_length += 1
            num_bullish_periods += 1
        elif df["low"].iloc[i] <= current_low - box_size:
            current_direction = -1
            last_low = current_low
            current_low = df["low"].iloc[i]
            df.loc[df.index[i], "pnf_low"] = current_low
            df.loc[df.index[i], "pnf_direction"] = current_direction
            if last_low is not None and current_low < last_low:
                df.loc[df.index[i], "pnf_dbb"] = 1  # double bottom breakdown
            column_heights.append(last_low - current_low)
            bullish_run_lengths.append(current_bullish_run_length)
            current_bullish_run_length = 0
            num_bearish_periods += 1
        elif (
            current_direction == 1
            and df["low"].iloc[i] <= current_high - box_size * reversal
        ):
            current_direction = -1
            last_low = current_low
            current_low = df["low"].iloc[i]
            df.loc[df.index[i], "pnf_low"] = current_low
            df.loc[df.index[i], "pnf_direction"] = current_direction
            df.loc[df.index[i], "pnf_num_bullish_to_bearish_reversals"] += 1
            column_heights.append(last_low - current_low)
            bullish_run_lengths.append(current_bullish_run_length)
            current_bullish_run_length = 0
            num_bearish_periods += 1
        elif (
            current_direction == -1
            and df["high"].iloc[i] >= current_low + box_size * reversal
        ):
            current_direction = 1
            last_high = current_high
            current_high = df["high"].iloc[i]
            df.loc[df.index[i], "pnf_high"] = current_high
            df.loc[df.index[i], "pnf_direction"] = current_direction
            df.loc[df.index[i], "pnf_num_bearish_to_bullish_reversals"] += 1
            column_heights.append(current_high - last_high)
            current_bullish_run_length += 1
            num_bullish_periods += 1

        df.loc[df.index[i], "pnf_num_reversals"] = (
            df.loc[df.index[i], "pnf_num_bearish_to_bullish_reversals"]
            + df.loc[df.index[i], "pnf_num_bullish_to_bearish_reversals"]
        )
        df.loc[df.index[i], "pnf_avg_column_height"] = (
            np.nan if not column_heights else np.mean(column_heights)
        )
        df.loc[df.index[i], "pnf_max_column_height"] = (
            np.nan if not column_heights else np.max(column_heights)
        )
        df.loc[df.index[i], "pnf_min_column_height"] = (
            np.nan if not column_heights else np.min(column_heights)
        )
        df.loc[df.index[i], "pnf_std_column_height"] = (
            np.nan if not column_heights else np.std(column_heights)
        )
        df.loc[df.index[i], "pnf_bullish_percentage"] = (
            np.nan
            if not num_bullish_periods
            else num_bullish_periods / (num_bullish_periods + num_bearish_periods)
        )
        df.loc[df.index[i], "pnf_bearish_percentage"] = (
            np.nan
            if not num_bearish_periods
            else num_bearish_periods / (num_bullish_periods + num_bearish_periods)
        )
        df.loc[df.index[i], "pnf_avg_bullish_run_length"] = (
            np.nan if not bullish_run_lengths else np.mean(bullish_run_lengths)
        )

    # Calculate the mid point as pnf_close
    df["pnf_close"] = (df["high"] + df["low"]) / 2

    # Calculate RSI
    df["pnf_RSI"] = talib.RSI(df["pnf_close"], timeperiod=8)

    # Calculate MACD
    macd, macdsignal, macdhist = talib.MACD(
        df["pnf_close"], fastperiod=8, slowperiod=17, signalperiod=5
    )
    df["pnf_MACD"] = macd

    # Calculate VWAP
    df["pnf_VWAP"] = np.cumsum(df["volume"] * df["pnf_close"]) / np.cumsum(df["volume"])

    # Calculate SMA
    df["pnf_SMA"] = talib.SMA(df["pnf_close"], timeperiod=20)

    # Calculate EMA
    df["pnf_EMA"] = talib.EMA(df["pnf_close"], timeperiod=20)

    return df


def create_features(df):
    ## DIFFERENCES ##

    # Price differences
    df["price_diff"] = df["close"].diff()
    df["op_cl_diff"] = df["open"] - df["close"]

    # Moving averages
    df["ma_5"] = df["close"].rolling(window=5).mean()
    df["ma_10"] = df["close"].rolling(window=10).mean()

    # Price percentage change
    df["pct_change"] = df["close"].pct_change()

    # RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["rsi"] = 100 - 100 / (1 + rs)

    # Other popular difference features
    for i in range(1, 35):
        df["diff_{}".format(i)] = df["close"].diff(i)

    sma = df["close"].rolling(window=20).mean()
    std = df["close"].rolling(window=20).std()
    df["upper_band"] = sma + (2 * std)
    df["lower_band"] = sma - (2 * std)

    highest_high = df["high"].rolling(window=14).max()
    lowest_low = df["low"].rolling(window=14).min()
    df["williams_r"] = (highest_high - df["close"]) / (highest_high - lowest_low) * -100

    df["obv"] = (np.sign(df["close"].diff()) * df["volume"]).fillna(0).cumsum()

    tp = (df["high"] + df["low"] + df["close"]) / 3
    sma = tp.rolling(window=20).mean()
    mean_deviation = tp.rolling(window=20).apply(
        lambda x: np.mean(np.abs(x - x.mean()))
    )
    df["cci"] = (tp - sma) / (0.015 * mean_deviation)

    true_range = pd.concat(
        [
            df["high"] - df["low"],
            (df["high"] - df["close"].shift()).abs(),
            (df["close"].shift() - df["low"]).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["atr"] = true_range.rolling(window=14).mean()

    money_flow_vol = (
        ((df["close"] - df["low"]) - (df["high"] - df["close"]))
        / (df["high"] - df["low"])
        * df["volume"]
    )
    cmf = (
        money_flow_vol.rolling(window=20).sum() / df["volume"].rolling(window=20).sum()
    )
    df["cmf"] = cmf

    ema_12 = df["close"].ewm(span=12).mean()
    ema_26 = df["close"].ewm(span=26).mean()
    ppo = (ema_12 - ema_26) / ema_26 * 100
    df["ppo"] = ppo

    ### LAG ###

    # Lag closing prices
    for i in range(1, 11):
        df["lag_close_{}".format(i)] = df["close"].shift(i)

    # Lag daily returns
    returns = df["close"].pct_change()
    for i in range(1, 6):
        df["lag_return_{}".format(i)] = returns.shift(i)

    # Lag high and low prices
    for i in range(1, 4):
        df["lag_high_{}".format(i)] = df["high"].shift(i)
        df["lag_low_{}".format(i)] = df["low"].shift(i)

    # Historical volatility
    df["hist_volatility_10"] = returns.rolling(window=10).std()
    df["hist_volatility_20"] = returns.rolling(window=20).std()
    df["hist_volatility_30"] = returns.rolling(window=30).std()

    # Previous day's RSI value
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - 100 / (1 + rs)
    df["lag_rsi_1"] = rsi.shift(1)

    ### ROLLING ###

    # Moving averages
    df["ma_5"] = df["close"].rolling(window=5).mean()
    df["ma_10"] = df["close"].rolling(window=10).mean()
    df["ma_20"] = df["close"].rolling(window=20).mean()

    # Exponential moving averages
    df["ema_5"] = df["close"].ewm(span=5).mean()
    df["ema_10"] = df["close"].ewm(span=10).mean()
    df["ema_20"] = df["close"].ewm(span=20).mean()

    # Bollinger Bands
    sma = df["close"].rolling(window=20).mean()
    std = df["close"].rolling(window=20).std()
    df["upper_band"] = sma + (2 * std)
    df["lower_band"] = sma - (2 * std)

    # Rate of Change (ROC)
    df["roc_5"] = df["close"].pct_change(periods=5)
    df["roc_10"] = df["close"].pct_change(periods=10)

    # Standard deviation
    df["std_5"] = df["close"].rolling(window=5).std()
    df["std_10"] = df["close"].rolling(window=10).std()

    # Average True Range (ATR)
    true_range = pd.concat(
        [
            df["high"] - df["low"],
            (df["high"] - df["close"].shift()).abs(),
            (df["low"] - df["close"].shift()).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["atr_14"] = true_range.rolling(window=14).mean()

    # Keltner Channels
    middle_line = df["close"].rolling(window=20).mean()
    upper_keltner = middle_line + 2 * df["atr_14"]
    lower_keltner = middle_line - 2 * df["atr_14"]
    df["upper_keltner"] = upper_keltner
    df["lower_keltner"] = lower_keltner

    # RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["rsi"] = 100 - 100 / (1 + rs)

    ### RATIO ###

    # Calculate some basic features first
    returns = df["close"].pct_change()
    ema_12 = df["close"].ewm(span=12).mean()
    ema_26 = df["close"].ewm(span=26).mean()

    # 1. Price-to-moving-average ratios
    df["price_to_ma_5"] = df["close"] / df["ma_5"]
    df["price_to_ma_10"] = df["close"] / df["ma_10"]
    df["price_to_ma_20"] = df["close"] / df["ma_20"]

    # 2. Price-to-EMA ratios
    df["price_to_ema_5"] = df["close"] / df["ema_5"]
    df["price_to_ema_10"] = df["close"] / df["ema_10"]
    df["price_to_ema_20"] = df["close"] / df["ema_20"]

    # 3. EMA-to-moving-average ratios
    df["ema_to_ma_5"] = df["ema_5"] / df["ma_5"]
    df["ema_to_ma_10"] = df["ema_10"] / df["ma_10"]
    df["ema_to_ma_20"] = df["ema_20"] / df["ma_20"]

    # 4. EMA-to-EMA ratios
    df["ema5_to_ema10"] = df["ema_5"] / df["ema_10"]
    df["ema5_to_ema20"] = df["ema_5"] / df["ema_20"]
    df["ema10_to_ema20"] = df["ema_10"] / df["ema_20"]

    # 5. Moving-average-to-moving-average ratios
    df["ma5_to_ma10"] = df["ma_5"] / df["ma_10"]
    df["ma5_to_ma20"] = df["ma_5"] / df["ma_20"]
    df["ma10_to_ma20"] = df["ma_10"] / df["ma_20"]

    # 6. Price-to-Bollinger-Band ratios
    df["price_to_upper_band"] = df["close"] / df["upper_band"]
    df["price_to_lower_band"] = df["close"] / df["lower_band"]

    # 7. Price-to-Keltner-Channel ratios
    df["price_to_upper_keltner"] = df["close"] / df["upper_keltner"]
    df["price_to_lower_keltner"] = df["close"] / df["lower_keltner"]

    # 8. Price-to-previous-day-high ratio
    df["price_to_prev_high"] = df["close"] / df["high"].shift(1)

    # 9. Price-to-previous-day-low ratio
    df["price_to_prev_low"] = df["close"] / df["low"].shift(1)

    # 10. RSI-to-moving-average ratio
    df["rsi_to_ma_5"] = df["rsi"] / df["ma_5"]

    ### CORRELATION ####

    # Calculate some basic features first
    returns = df["close"].pct_change()
    volume_change = df["volume"].pct_change()

    # 1. Correlation with returns and volume (short-term)
    df["return_volume_corr_5"] = returns.rolling(window=5).corr(df["volume"])
    df["return_volume_corr_10"] = returns.rolling(window=10).corr(df["volume"])
    df["return_volume_corr_20"] = returns.rolling(window=20).corr(df["volume"])

    # 2. Correlation with returns and volume (medium-term)
    df["return_volume_corr_50"] = returns.rolling(window=50).corr(df["volume"])
    df["return_volume_corr_100"] = returns.rolling(window=100).corr(df["volume"])

    # 3. Correlation with price and volume
    df["price_volume_corr_5"] = df["close"].rolling(window=5).corr(df["volume"])
    df["price_volume_corr_10"] = df["close"].rolling(window=10).corr(df["volume"])
    df["price_volume_corr_20"] = df["close"].rolling(window=20).corr(df["volume"])

    # 4. Correlation with price and volume (medium-term)
    df["price_volume_corr_50"] = df["close"].rolling(window=50).corr(df["volume"])
    df["price_volume_corr_100"] = df["close"].rolling(window=100).corr(df["volume"])

    # 5. Price / volume
    df["price_to_volume"] = df["close"] / df["volume"]

    # 6. Volume Relative Strength Index (VRSI)
    delta_vol = df["volume"].diff()
    gain_vol = delta_vol.where(delta_vol > 0, 0)
    loss_vol = -delta_vol.where(delta_vol < 0, 0)
    avg_gain_vol = gain_vol.rolling(window=14).mean()
    avg_loss_vol = loss_vol.rolling(window=14).mean()
    rs_vol = avg_gain_vol / avg_loss_vol
    df["vrsi"] = 100 - 100 / (1 + rs_vol)

    # 7. Change in volume
    df["volume_change"] = volume_change

    # 8. Moving averages of volume
    df["volume_ma_5"] = df["volume"].rolling(window=5).mean()
    df["volume_ma_10"] = df["volume"].rolling(window=10).mean()
    df["volume_ma_20"] = df["volume"].rolling(window=20).mean()

    # 9. Standard deviation of volume
    df["volume_std_5"] = df["volume"].rolling(window=5).std()
    df["volume_std_10"] = df["volume"].rolling(window=10).std()
    df["volume_std_20"] = df["volume"].rolling(window=20).std()

    # 10. Ratio of volume to moving average volume
    df["volume_to_ma_volume"] = df["volume"] / df["volume_ma_20"]

    ### EXTRA ###

    # Calculate necessary base features
    returns = df["close"].pct_change()
    delta = df["close"].diff()

    # close_change
    df["close_change"] = df["close"].diff()

    # high_pct
    df["high_pct"] = df["high"].pct_change()

    # close_change_roll5
    df["close_change_roll5"] = df["close_change"].rolling(window=5).mean()

    # RSI_14_roll5
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - 100 / (1 + rs)
    df["RSI_14_roll5"] = rsi.rolling(window=5).mean()

    # ATR_14_roll5
    true_range = pd.concat(
        [
            df["high"] - df["low"],
            (df["high"] - df["close"].shift()).abs(),
            (df["low"] - df["close"].shift()).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["ATR_14_roll5"] = true_range.rolling(window=14).mean().rolling(window=5).mean()

    # volume_roll5
    df["volume_roll5"] = df["volume"].rolling(window=5).mean()

    # high_pct_roll5
    df["high_pct_roll5"] = df["high_pct"].rolling(window=5).mean()

    # volatility_5
    df["volatility_5"] = df["close"].rolling(window=5).std()

    # price_ema5
    df["price_ema5"] = df["close"].ewm(span=5).mean()

    # volume_ema5
    df["volume_ema5"] = df["volume"].ewm(span=5).mean()

    # price_to_ema5
    df["price_to_ema5"] = df["close"] / df["price_ema5"]

    # volume_change_roll5
    df["volume_change_roll5"] = volume_change.rolling(window=5).mean()

    # avg_vol_last_100
    df["avg_vol_last_100"] = df["volume"].rolling(window=100).mean()

    # turnover
    df["turnover"] = df["volume"] * df["close"]
    # ema5 rolling
    df["volume_ema5"] = df["volume"].ewm(span=5).mean()

    bollinger_5 = ta.bbands(df["close"], length=5)
    df = pd.concat([df, bollinger_5.add_suffix("_5")], axis=1)

    df["hollow_candle"] = np.where(df["close"] > df["open"], 1, 0)

    force_index = ForceIndexIndicator(df["close"], df["volume"], 13)
    df["force_index"] = force_index.force_index()

    df["equivolume_width"] = (df["volume"] - df["volume"].min()) / (
        df["volume"].max() - df["volume"].min()
    )

    return df


def dynamic_line_break(df, initial_bricks=3):
    df = df.copy()
    df["line_break"] = 0  # Initialize the column

    # Fill in the first few bricks
    for i in range(initial_bricks):
        df.loc[i, "line_break"] = df.loc[i, "close"]

    for i in range(initial_bricks, len(df)):
        if df.loc[i, "close"] > df.loc[i - 1, "line_break"]:
            df.loc[i, "line_break"] = df.loc[i, "close"]
        elif df.loc[i, "close"] < df.loc[i - 1, "line_break"]:
            df.loc[i, "line_break"] = df.loc[i, "close"]
        else:
            df.loc[i, "line_break"] = df.loc[i - 1, "line_break"]

    return df


def timeseries_cv_score(X, y, n_splits):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    f1_scores = []
    auc_scores = []  # list to store ROC AUC scores for each split
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Define LSTM model
        model = Sequential()
        model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dense(1, activation="sigmoid"))  # because of binary classification

        model.compile(
            loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        # Train the model
        model.fit(X_train, y_train, epochs=10, verbose=0)

        # Make predictions on the test set
        y_pred = model.predict(X_test).ravel()

        # Calculate F1 score of the model on the test set
        f1 = f1_score(y_test, (y_pred > 0.5).astype("int32"))
        f1_scores.append(f1)

        # Calculate ROC AUC score of the model on the test set
        auc = roc_auc_score(y_test, y_pred)
        auc_scores.append(auc)

    return np.mean(f1_scores), np.mean(auc_scores)

## Globals

In [214]:
# Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# Prepare TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
# Load the data
data_path = "../../../data/kc/btc/raw/kc_btc_15min.csv"

## Preprocessing

In [215]:
# Load and preprocess the data
df = load_data(data_path)
df = preprocess_data(df)

# Timestamp conversion and index setting
df["time_unix"] = df["time"]
df["time"] = pd.to_datetime(df["time"], unit="s")  # Assuming 'time' is in seconds
df.set_index("time", inplace=True)
df = df.loc[~df.index.duplicated(keep="first")]

# Technical Analysis features
df.ta.strategy("all")

# Check your results and exclude as necessary.
df.ta.strategy(fast=10, slow=50, verbose=True)

# Heiken Ashi 100%
df = add_heiken_ashi_features(df)

## RENKO
# CONSIDER CHANGING multiplyer to .5 or even 1
renko_df = compute_renko(df)
df = add_renko_features(df, renko_df)

## Three Line Break ##

df_tlb = three_line_break(df)
direction_reversals = (
    df_tlb["tlb_direction"].shift(1) != df_tlb["tlb_direction"]
).sum()
print("The number of direction reversals:", direction_reversals)


# Create the TLB dataframe
tlb_df = three_line_break(df)
# Add TLB features
tlb_df = add_tlb_features(tlb_df)
# Join TLB dataframe with original dataframe
df = df.join(tlb_df.set_index("time_unix"), on="time_unix", rsuffix="_tlb")

# Additional features
df = create_features(df)

# Now extract additional date information from the original dataframe
df["minute"] = df.index.minute
df["hour"] = df.index.hour
df["day"] = df.index.day
df["month"] = df.index.month

df["pnf_ATR"] = custom_atr(df, 14)
df["pnf_ATR"] = df["pnf_ATR"].fillna(method="ffill").fillna(method="bfill")
atr_value = df["pnf_ATR"].iloc[-1]
box_size = atr_value / 6

# Use the PNF function 100%
pnf_df = pnf(df, box_size, atr_value / 2)
direction_changes = pnf_df["pnf_direction"].ne(0).sum()
print(f"Number of direction changes: {direction_changes}")

## KAGI ##
kagi_df = kagi(df)
# Add Kagi features
kagi_df = add_kagi_features(df, kagi_df)
df = df.join(kagi_df.set_index("time_unix"), on="time_unix")

# Sanity check. Make sure all the columns and types
# print(df.columns)

# Forward Fill
df.ffill(inplace=True)

# Backward Fill
df.bfill(inplace=True)

# Identify constant columns
constant_columns = [col for col in df.columns if df[col].nunique() == 1]
print("Constant columns: ", constant_columns)

# Drop constant columns
df = df.drop(columns=constant_columns)

# Check if the data has any NaN values and handle them
if df.isna().sum().sum() > 0:
    nan_cols = df.columns[df.isna().any()].tolist()
    print("Columns with NaN: ", nan_cols)
    df = df.fillna(method="ffill")  # or use any other method you deem appropriate

# Check if data has any Infinite values and handle them
if np.isinf(df).values.sum() > 0:
    inf_cols = df.columns.to_series()[np.isinf(df).any()].tolist()
    print("Columns with Inf: ", inf_cols)
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(method="ffill")  # or use any other method you deem appropriate


# Finding problem features for standard scalar
non_num_features = df.select_dtypes(exclude=["int32", "int64", "float32", "float64"])
print("Fix these features:\n")
for col, dtype in non_num_features.dtypes.items():
    print(f"{col}: {dtype}")

# Select numeric columns which need to be scaled
do_not_scale_columns = [
    "time_unix",
    "minute",
    "hour",
    "day",
    "month",
    "color_change",
]
scaler = StandardScaler()
for col in df.columns:
    if col not in do_not_scale_columns:
        df[col] = scaler.fit_transform(df[[col]])

X = df.drop("color_change", axis=1)
y = df["color_change"]

# duplicate features
duplicated_features = df.columns.duplicated()
print("Duplicate Features: ", df.columns[duplicated_features])
# Total features
print("Total features in DataFrame: ", df.shape[1])
# df.tail(100)

131it [00:02, 52.34it/s]


[+] Strategy: All
[i] Indicator arguments: {'fast': 10, 'slow': 50, 'append': True}
[i] Excluded[12]: above, above_value, below, below_value, cross, cross_value, long_run, short_run, td_seq, tsignals, vp, xsignals
[i] Multiprocessing 131 indicators with 3 chunks and 12/12 cpus.


131it [00:02, 48.84it/s]


[i] Total indicators: 131
[i] Columns added: 30
[i] Last Run: Monday June 26, 2023, NYSE: 20:18:56, Local: 0:18:56 Pacific Daylight Time, Day 177/365 (48.00%)
The number of direction reversals: 1719
Number of direction changes: 1199
Constant columns:  ['CDL_3BLACKCROWS', 'CDL_3STARSINSOUTH', 'CDL_ABANDONEDBABY', 'CDL_BREAKAWAY', 'CDL_CONCEALBABYSWALL', 'CDL_COUNTERATTACK', 'CDL_INNECK', 'CDL_KICKING', 'CDL_KICKINGBYLENGTH', 'CDL_LADDERBOTTOM', 'CDL_MATHOLD', 'CDL_ONNECK', 'CDL_TASUKIGAP', 'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS']
Fix these features:

Duplicate Features:  Index([], dtype='object')
Total features in DataFrame:  572


Feature Selection

In [216]:
iteration = 1
max_iterations = 8
final_df = pd.DataFrame(columns=["feature", "score", "p_value", "iteration", "process"])
scaler = StandardScaler()
X_all = X.copy()  # Full set of features

# Use the function before UFS and RFE
check_data_balance(y)

while iteration <= max_iterations and X_all.shape[1] > 0:
    # Scale the data
    X_scaled = scaler.fit_transform(X_all)

    # Apply Univariate Feature Selection
    selector = SelectKBest(score_func=f_classif, k="all")
    selector.fit(X_scaled, y)

    # Get columns to keep
    cols = selector.get_support(indices=True)
    features_df_new = X_all.iloc[:, cols]

    # Create a dataframe with the features, their scores and p-values for the current iteration
    features = features_df_new.columns
    scores = selector.scores_
    p_values = selector.pvalues_
    current_df = pd.DataFrame(
        {
            "feature": features,
            "score": scores,
            "p_value": p_values,
            "iteration": iteration,
            "process": "UFS",
        }
    )

    # Append current dataframe to the final dataframe
    final_df = pd.concat([final_df, current_df], ignore_index=True)

    # Only keep features with p-value less than 0.05 for the next iteration
    X_all = features_df_new.loc[:, p_values < 0.05]

    iteration += 1

final_df["reoccurrence"] = final_df.groupby("feature")["feature"].transform("count")
final_df.to_csv("selected_features.csv", index=False)

Class 0: 3766 samples (46.73%)
Class 1: 4293 samples (53.27%)


RFE

In [221]:
# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)  # Adjust the number of splits as necessary

n_features_to_select = [88, 66, 44, 22]
output_df = pd.DataFrame()

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    for i in n_features_to_select:
        selector = RFE(
            estimator=RandomForestClassifier(), n_features_to_select=i, step=1
        )
        selector = selector.fit(X_train, y_train)
        y_pred = selector.predict(X_test)

        feature_importance = selector.estimator_.feature_importances_
        selected_features = X_train.columns[selector.support_]

        for j, feature in enumerate(selected_features):
            temp_df = pd.DataFrame(
                {
                    "features": [feature],
                    "f1_score": [f1_score(y_test, y_pred)],
                    "feature_importance": [feature_importance[j]],
                    "iteration": [i],
                    "process": ["RFE"],
                }
            )
            output_df = pd.concat([output_df, temp_df], ignore_index=True)

# Count the occurrence of each feature
output_df["reoccurence"] = output_df.groupby("features")["features"].transform("count")

# Save the dataframe to a csv file
output_df.to_csv("rfe_selected_features.csv", index=False)

KeyboardInterrupt: 

## Univariate Feature Selection Process

In [218]:
# Re-scale the data to include the new feature
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Feature selection
selector = SelectKBest(score_func=f_classif, k="all")
selector.fit(X_scaled, y)


# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
features_df_new = X.iloc[:, cols]

# Store the scores and p-values of each feature in a dictionary
feature_scores_pvalues = {
    feature_name: (score, p_value)
    for feature_name, score, p_value in zip(
        X.columns, selector.scores_, selector.pvalues_
    )
}

# Sort the dictionary by score value in descending order and print the scores and p-values
for feature_name, values in sorted(
    feature_scores_pvalues.items(), key=lambda item: item[1][0], reverse=True
):
    print(f"{feature_name}: score={values[0]}, p_value={values[1]}")


# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)

# TimeSeries Cross-validation
tscv = TimeSeriesSplit(n_splits=5)

print(y.value_counts())

# Cross-validation
cv_scores = cross_val_score(log_reg, features_df_new, y, cv=tscv, scoring="f1")
print(f"\nLogistic Regression CV F1 score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, features_df_new, y, cv=tscv, scoring="f1")
print(f"Random Forest CV F1 score: {np.mean(cv_scores)}")

# Reshape input to be 3D [samples, timesteps, features]
X_array = X.values
X_reshaped = X_array.reshape((X_array.shape[0], 1, X_array.shape[1]))

# Call the function
mean_f1_score, mean_auc_score = timeseries_cv_score(X_reshaped, y.values, n_splits=5)
print(f"\nLSTM CV F1 score: {mean_f1_score}, LSTM CV AUC score: {mean_auc_score}")

print("\n", features_df_new.columns)

CDL_DOJI_10_0.1: score=9.310771093067425, p_value=0.0022855200847150536
lag_return_3: score=8.21725193351612, p_value=0.004160153632398433
price_to_prev_low: score=7.953180853307701, p_value=0.004812020758771848
CDL_LONGLEGGEDDOJI: score=7.900466256835382, p_value=0.004954135454008858
CDL_RICKSHAWMAN: score=6.829982600613045, p_value=0.008980676157158498
CDL_ENGULFING: score=6.153134552700921, p_value=0.013138204811504775
CDL_CLOSINGMARUBOZU: score=6.123655074636918, p_value=0.013358904196747422
LOGRET_1: score=5.591572392932741, p_value=0.01807065897578273
PCTRET_1: score=5.581321370219304, p_value=0.01817665387415499
pct_change: score=5.581321370219304, p_value=0.01817665387415499
SLOPE_1: score=5.039338687447538, p_value=0.02480496635170951
price_diff: score=5.039338687447538, p_value=0.02480496635170951
diff_1: score=5.039338687447538, p_value=0.02480496635170951
close_change: score=5.039338687447538, p_value=0.02480496635170951
CFO_9: score=4.953768605914391, p_value=0.02606142274

## Base Model

In [219]:
# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=500)

# Cross-validation
cv_scores = cross_val_score(log_reg, X, y, cv=tscv, scoring="roc_auc")

print(f"Logistic Regression CV ROC AUC score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, X, y, cv=tscv, scoring="roc_auc")

print(f"Random Forest CV ROC AUC score: {np.mean(cv_scores)}")

print("\n", X.columns)

Logistic Regression CV ROC AUC score: 0.5054162177583823


KeyboardInterrupt: 

## LSTM

In [ ]:
# Reshape input to be 3D [samples, timesteps, features]
X_array = X.values
X_reshaped = X_array.reshape((X_array.shape[0], 1, X_array.shape[1]))

# Call the function
mean_auc_score = timeseries_cv_score(X_reshaped, y.values, n_splits=5)
print(f"\nLSTM CV ROC AUC score: {mean_auc_score}")

42/42 [==============================] - 0s 844us/step

LSTM CV ROC AUC score: (0.6973535002726867, 0.5)
